In [0]:
from pyspark.sql.functions import col,regexp_replace,schema_of_json,from_json

In [0]:
df = spark.table("gizmobox.bronze.py_orders")

In [0]:
df_updates = df.withColumn("value",regexp_replace("value",'"order_date": (\\d{4}-\\d{2}-\\d{2})','"order_date": "$1"'))

In [0]:
df_schema = df_updates.select(schema_of_json(col("value")).alias("schema")).limit(1)

In [0]:
df_final = df_updates.select(from_json(col("value"),"STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>").alias("fixed_value"))

In [0]:
df_final.writeTo("gizmobox.silver.py_orders_json").createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_orders_json